In [1]:
from pytorch_pretrained_vit import ViT
from torchvision import transforms
from EEG_Image_Dataset import *
from torch import nn
import torch
import numpy as np
import glob
import pickle
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import tqdm
import time
import torch
from torchvision import transforms
from torch import nn
import numpy as np
import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import  torch.nn.functional as F
import cv2

In [2]:
class Image_Encoding(nn.Module):
    def __init__(self, backbone, num_labels):
        super().__init__()
        self.ViT = backbone
        self.Act = nn.ReLU(True)
        self.classification = nn.Linear(1000, 40)
        
    def forward(self, x):
        encoded_output = self.ViT(x)
        embedding = self.Act(encoded_output)
        classification_output = self.classification(encoded_output)
        
        return embedding, classification_output

In [3]:
Image_Encoder = ViT('B_16_imagenet1k', pretrained=True)

#Freeze pretrained weights except last two fc layers
for param in Image_Encoder.parameters():
    param.requires_grad = False
Image_Encoder.cuda()
print("Copied to CUDA")

Loaded pretrained weights.
Copied to CUDA


In [4]:
with open('/users/sbr/data/Image_data/image_path.pkl', 'rb') as f:
    image_path = pickle.load(f)
    
eeg_dataset = '/users/sbr/data/eeg_55_95_std.pth'
splits_path = '/users/sbr/data/block_splits_by_image_all.pth'
dataset = EEG_Image_Dataset(eeg_dataset, image_path)
loaders = {split: DataLoader(Splitter(dataset, split_path = splits_path, split_name = split), batch_size = 16,drop_last = True, shuffle = True) for split in ["train", "val", "test"]}

In [7]:
losses_per_epoch={"train":[], "val":[],"test":[]}
accuracies_per_epoch={"train":[],"val":[],"test":[]}
optimizer = torch.optim.Adam(Image_Encoder.parameters())

best_accuracy = 0
best_accuracy_val = 0
best_epoch = 0
EPOCHS = 20

predicted_labels = [] 
correct_labels = []

for epoch in range(1, EPOCHS+1):
    start = time.time()
    losses = {"train": 0, "val": 0, "test": 0}
    accuracies = {"train": 0, "val": 0, "test": 0}
    counts = {"train": 0, "val": 0, "test": 0}

    for split in ("train", "val", "test"):
        # Set network mode
        if split == "train":
            Image_Encoder.train()
            torch.set_grad_enabled(True)
        else:
            Image_Encoder.eval()
            torch.set_grad_enabled(False)
        # Process all split batches
        tqdm_gen = tqdm.tqdm(loaders[split])
        for i, (_, image, _, label) in enumerate(tqdm_gen):
            image = image.to("cuda") 
            label = label.to("cuda")
            output = Image_Encoder(image)

            # Compute loss
            loss = F.cross_entropy(output, label)
            losses[split] += loss.item()
            # Compute accuracy
            _,pred = output.data.max(1)
            correct = pred.eq(label.data).sum().item()
            accuracy = correct/image.data.size(0)   
            accuracies[split] += accuracy
            counts[split] += 1
            print(correct)
            # Backward and optimize
            if split == "train":
                optimizer.zero_grad()
                loss.requires_grad_(True)
                loss.backward()
                optimizer.step()

    # Print info at the end of the epoch
    if accuracies["val"]/counts["val"] >= best_accuracy_val:
        best_accuracy_val = accuracies["val"]/counts["val"]
        best_accuracy = accuracies["test"]/counts["test"]
        best_epoch = epoch
    
    
    TrL,TrA,VL,VA,TeL,TeA=  losses["train"]/counts["train"],accuracies["train"]/counts["train"],losses["val"]/counts["val"],accuracies["val"]/counts["val"],losses["test"]/counts["test"],accuracies["test"]/counts["test"]
    print('Time for Epoch{0}: {1:.2f}'.format(epoch, time.time() - start))
    print("Epoch {0}: TrL={1:.4f}, TrA={2:.4f}, VL={3:.4f}, VA={4:.4f}, TeL={5:.4f}, TeA={6:.4f}, TeA at max VA = {7:.4f} at epoch {8:d}".format(epoch,
                                                                                                         losses["train"]/counts["train"],
                                                                                                         accuracies["train"]/counts["train"],
                                                                                                         losses["val"]/counts["val"],
                                                                                                         accuracies["val"]/counts["val"],
                                                                                                         losses["test"]/counts["test"],
                                                                                                         accuracies["test"]/counts["test"],
                                                                                                         best_accuracy, best_epoch))
    
    losses_per_epoch['train'].append(TrL)
    losses_per_epoch['val'].append(VL)
    losses_per_epoch['test'].append(TeL)
    accuracies_per_epoch['train'].append(TrA)
    accuracies_per_epoch['val'].append(VA)
    accuracies_per_epoch['test'].append(TeA)

    if epoch%10 == 0:
                torch.save(Image_Encoder, '%s_epoch_%d.pth' % ('Image_Encoder', epoch))
            

  0%|                                           | 1/497 [00:00<04:10,  1.98it/s]

15


  0%|▏                                          | 2/497 [00:00<03:40,  2.24it/s]

14


  1%|▎                                          | 3/497 [00:01<03:30,  2.35it/s]

...

  3%|█▏                                        | 14/497 [00:05<03:25,  2.35it/s]

16


  3%|█▎                                        | 15/497 [00:06<03:32,  2.27it/s]

14


KeyboardInterrupt: 